In [105]:
import cv2  as cv
import datetime
import time

In [106]:
titles = ["Name", "Blood", "Experience(km)", "Accident"]

# People who allowed to drive the car
people = [["Ali Emre Kaya", "0+", 134, 0]]

In [107]:
# Load a trained face recognizer model
face_recognizer = cv.face.LBPHFaceRecognizer_create()
face_recognizer.read("face_trained.yml")

In [108]:
# Load OpenCV's pre-trained Haar cascade for face detection
haar_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [129]:
def take_photo():
    cap = cv.VideoCapture(0)

    # Give enough time to camera for opening 
    time.sleep(1)
    ret, frame = cap.read()   

    # Save frame in current_driver folder as jpg file 
    count = 0
    cv.imwrite("current_driver/frame%d.jpg" % count, frame)
    image_name = "frame%d" % count
    
    img = cv.imread(f"current_driver/{image_name}.jpg")
    pure_img = img

    # Get the height and width of the image
    height, width = img.shape[:2]

    # Convert the image to grayscale for better analysis
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Detect faces using the Haar cascade
    faces_rect = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)

    if(len(faces_rect) == 0):
        print("FAIL, no face detected.")

    else:
        # Selecting the biggest face
        # (assuming that will be the inividual)
        biggest_rect = -1
        xB = 0
        yB = 0
        wB = 0
        hB = 0
        for(x, y, w, h) in faces_rect:
            new_rect = abs(w)
            if(biggest_rect < new_rect):
                biggest_rect = new_rect
                xB = x
                yB = y
                wB = w
                hB = h

        # Indexes of biggest face
        faces_rect = [[xB, yB, wB, hB]]

        # Controlling the confidence
        confidence_ratio = 200

        # Rectangle and Text
        for (x, y, w, h) in faces_rect:
            # Extract the region of interest (ROI) for face recognition
            faces_roi = gray[y:y+h, x:x+w]

            # Perform face recognition to determine the person and confidence level
            label, not_confidence = face_recognizer.predict(faces_roi)
            print(f"{people[label][0]} : {not_confidence}")

            if(not_confidence < confidence_ratio):
                confidence_ratio = not_confidence


            # Define font properties for displaying text, general
            font_scale = height / 800.0
            thickness = max(1, int(height / 300))
            not_confidence_size = cv.getTextSize(str(not_confidence), cv.FONT_HERSHEY_SIMPLEX, font_scale, thickness=2)[0]
            start_x_conf = int((x + (w/2)) - not_confidence_size[1] / 2)

            # Driving permission given
            if(confidence_ratio < 22):
                # Define font properties for displaying text when if condition true
                text = people[label][0]
                text_size = cv.getTextSize(str(text), cv.FONT_HERSHEY_SIMPLEX, font_scale, thickness=2)[0]
                start_x = int((x + (w/2)) - text_size[0] / 2)
                # Display the name above and below the detected face
                cv.putText(img, str(text), (start_x, y - int(height/50)), cv.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), thickness=2)
                cv.putText(img, str(int(not_confidence)), (start_x_conf, y + h + not_confidence_size[1] + int(height/50)), cv.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), thickness=2)

                # Draw a rectangle around the detected face
                cv.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), thickness=2)

                # Save image for better confidence (I'M NOT SURE ON THIS PART) that may be Overfitting
                current_time = datetime.datetime.now()
                timestamp = current_time.strftime("%Y%m%d_%H%M%S")
                output_path = f"dataset/{people[label][0]}/{timestamp}.jpg"
                cv.imwrite(output_path, pure_img)

                # Save frame in past_drives permanently as succesful
                current_time = datetime.datetime.now()
                timestamp = current_time.strftime("%Y%m%d_%H%M%S")
                output_path = f"past_drives/{timestamp}_YES.jpg"
                cv.imwrite(output_path, img)

                print(f"Allowed to drive, {people[label][0]} can start the car.")

                print("----------------------------------------------")
                print(f"{people[label][0]}'s Informations: ")
                for i in range(1, len(people[label])):
                    print(f"{titles[i]} : {people[label][i]}")
            
            else:
                # Define font properties for displaying text when if condition false
                text = "unknown"
                text_size = cv.getTextSize(str(text), cv.FONT_HERSHEY_SIMPLEX, font_scale, thickness=2)[0]
                start_x = int((x + (w/2)) - text_size[0] / 2)
                # Display the name above and below the detected face
                cv.putText(img, str(text), (start_x, y - int(height/50)), cv.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 255), thickness=2)
                cv.putText(img, str(int(not_confidence)), (start_x_conf, y + h + not_confidence_size[1] + int(height/50)), cv.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 255), thickness=2)

                # Draw a rectangle around the detected face
                cv.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), thickness=2)

                # Save frame in past_drives permanently as unsuccesful
                current_time = datetime.datetime.now()
                timestamp = current_time.strftime("%Y%m%d_%H%M%S")
                output_path = f"past_drives/{timestamp}_NO.jpg"
                cv.imwrite(output_path, img)

                print("Not allowed to drive.")

    
    # Permission to access camera
    cap.release()

    # Close the camera
    cv.destroyAllWindows()
    

In [130]:
take_photo()

Ali Emre Kaya : 15.863760654772824
Allowed to drive, Ali Emre Kaya can start the car.
----------------------------------------------
Ali Emre Kaya's Informations: 
Blood : 0+
Experience(km) : 134
Accident : 0
